In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#| code-fold: false
# IMPORTING LIBRARIES
# --------------------

# AstroML
from astroML.datasets import fetch_LINEAR_sample
from astropy.coordinates import SkyCoord
import astropy.units as u

# Basic libraries
import pickle
import os
import sys
from tqdm import tqdm

# Plotting
import seaborn as sns
from matplotlib import pyplot as plt
from matplotlib import ticker
import matplotlib.colors as mcolors
from matplotlib.font_manager import FontProperties

# DataFrame analysis
import pandas as pd

# Math libraries
import numpy as np
import scipy as sc
from scipy.stats import norm

# CONFIGURATION
# -------------
sns.set_theme() # setting the theme for plotting
sys.path.insert(0,'../src/')
np.random.seed(42)

# configuring plotting colors
colors = ['#5F6372', '#79A8A4', '#B2AD8F', '#92A186', '#AD8082']
blue = '#5F6372'
turqoise = '#79A8A4'
light_green = '#B2AD8F'
green = '#92A186'
pink = '#AD8082'
muted_colors = sns.set_palette(sns.color_palette(colors))

# configuring fonts for plotting
font = FontProperties()
font.set_family('avenir')
font.set_name('Big Caslon')
font.set_style('normal')
font.set_size('xx-large')


%matplotlib inline  

In [ ]:
#| code-fold: false
# Importing custom libraries
# ----------------------------
sys.path.insert(0,'../src/')
from BE_plotting import*
#from RR_lyrae_params import*
from blazhko_analysis import*

## Importing data

The first step is to import the `LINEAR` and `ZTF` data, as well as the data calculated in the previous notebook.

In [ ]:
#| code-fold: false
# DATA
dataLINEAR = fetch_LINEAR_sample(data_home='../inputs') # fetching the data from astroML data library
end = 'FINAL_FINAL_FINAL_complete'

# DATA from previous notebook
lc_analysis = pd.read_csv("../outputs/LC_ANALYSIS_"+end+".csv")
rrlyrae = pd.read_csv('../outputs/Lrrlyr_unprocessed.csv')
periodogr = pickle.load(open("../outputs/periodograms_"+end+".pkl", "rb"))
fits = np.load("../outputs/fits_"+end+".npy", allow_pickle=True)
ztf_data = np.load("../outputs/ztf_data_filter"+end+".npy", allow_pickle=True)

In [ ]:
lc_analysis = lc_analysis.loc[lc_analysis['NdataZTF']>40]
lc_analysis = lc_analysis.reset_index(drop=True)
print(lc_analysis.shape)
lc_analysis.head()

### Plotting a light curve as an example

In [ ]:
order = 0
LiNeAr_ids = lc_analysis['LINEAR id'].to_numpy()
total_num = len(lc_analysis['LINEAR id'].to_numpy())
Lid = LiNeAr_ids[order]
lc_data_order = dataLINEAR.get_light_curve(Lid)
time, mag, magerr = lc_data_order.T
for n, j in enumerate(fits):
    if j[0]==Lid:
        break

L1 = fits[n][1][0]

fig, ax = plt.subplots(1,2, figsize=(14,5))   
#fig.suptitle('STAR '+str(order+1)+' from '+str(total_num), fontsize=30, fontproperties=font)
fig.set_facecolor('white')

#--------
ax[0].errorbar(time, mag, magerr, fmt='.k', ecolor='gray',lw=1, ms=4, capsize=1.5,alpha=0.3)
ax[0].set_xlabel('Vrijeme (dani)',fontproperties=font)
ax[0].set_ylabel('Magnituda', fontproperties=font)
fig.suptitle('LINEAR objekt {0}'.format(Lid), fontproperties=font)
ax[0].invert_yaxis()

# -----

ax[1].set_xlabel('fit LINEAR svjetlosne krivulje s najboljim periodom', fontproperties=font, fontsize=14)
ax[1].set_ylabel('LINEAR normalizirana svjetlosna krivulja', fontproperties=font, fontsize=14)
ax[1].set_xlim(-0.1, 1.1)
ax[1].set_ylim(1.3, -0.3)
# data
xx, yy, zz = sort3arr(L1['dataPhasedTime'], L1['dataTemplate'], L1['dataTemplateErr'])
ax[1].errorbar(xx, yy, zz, fmt='.k', ecolor='gray', lw=1, ms=4, capsize=1.5, alpha=0.3)
# fit for Plinear
ax[1].plot(L1['modelPhaseGrid'], L1['modTemplate'], pink, markeredgecolor=pink, lw=1, fillstyle='top', linestyle='dashed')

plt.savefig("../images/primjer_svkrivulje_sve.png", dpi=600)

In [ ]:
lc_analysis.head()

# Determination of BE candidates

(write up entire explanation of the algorithm)

In [ ]:
blazhko_can = pd.DataFrame(())
blazhko_can = blazhko_determine(lc_analysis, blazhko_can)
blazhko_can.to_csv("../outputs/blazhko_can"+end+".csv", index=False)

In [ ]:
blazhko_can.shape

# Determining vallidity of BE candidates

In order to make visual interpretation easier and more robust, we are dividing our BE candidates into smaller tables. The categories are:
- LINEAR periodogram parameter satisfied
- ZTF periodogram parameter satisfied
- Chi2 parameter satisfied
    - ZTF Chi2 parameter
    - LINEAR Chi2 parameter

The last three are based on the `ChiType` for determining LINEAR or ZTF parameter satisfaction. The periodogram parameter is determined by the `IndicatorType` column.

# Visual inspection of every category

In [ ]:
be_other = blazhko_can[(blazhko_can['IndicatorType']!= 'Z') & (blazhko_can['IndicatorType']!= 'L')&(blazhko_can['ChiType']!='Z')&(blazhko_can['ChiType']!='L')]
be_other = be_other.reset_index(drop=True)
print(be_other.shape)
be_other.head()

In [ ]:
BE_candidates_ztf_per = category_analysis(blazhko_can, fits, periodogr, ztf_data, dataLINEAR, 'ztf_periodogram',id_list=None,parameter='IndicatorType', value='Z')

In [ ]:
blazhko_analyzer = BE_candidates_ztf_per.get_save_data()
end = 'ztf_periodogram'
blazhko_analyzer.to_csv("../outputs/blazhko_list"+end+".csv", index=False)

In [ ]:
blazhko_can_ztf = pd.read_csv('../outputs/blazhko_list'+end+'.csv')
BE_candidates_ztf_per = category_analysis(blazhko_can_ztf, fits, periodogr, ztf_data, dataLINEAR, 'ztf_periodogram',id_list=None,parameter=None, value=None)

In [ ]:
blazhko_analyzer = BE_candidates_ztf_per.get_save_data()
end = 'ztf_periodogram_final'
blazhko_analyzer.to_csv("../outputs/blazhko_list"+end+".csv", index=False)

---

In [ ]:
BE_candidates_linear_per = category_analysis(blazhko_can, fits, periodogr, ztf_data, dataLINEAR, 'linear_periodogram',id_list=None,parameter='IndicatorType', value='L')

In [ ]:
blazhko_analyzer = BE_candidates_linear_per.get_save_data()
end = 'linear_periodogram'
blazhko_analyzer.to_csv("../outputs/blazhko_list"+end+".csv", index=False)

In [ ]:
blazhko_can_linearper = pd.read_csv('../outputs/blazhko_list'+end+'.csv')
BE_candidates_linear_per = category_analysis(blazhko_can_linearper, fits, periodogr, ztf_data, dataLINEAR, 'ztf_periodogram',id_list=None,parameter=None, value=None)

In [ ]:
blazhko_analyzer = BE_candidates_linear_per.get_save_data()
end = 'linear_periodogram_final'
blazhko_analyzer.to_csv("../outputs/blazhko_list"+end+".csv", index=False)

---

In [ ]:
BE_candidates_ztf_chi = category_analysis(blazhko_can, fits, periodogr, ztf_data, dataLINEAR, 'ztf_chi',id_list=None,parameter='ChiType', value='Z')

In [ ]:
blazhko_analyzer = BE_candidates_ztf_chi.get_save_data()
end = 'ztf_chi'
blazhko_analyzer.to_csv("../outputs/blazhko_list"+end+".csv", index=False)

In [ ]:
blazhko_can_ztfchi = pd.read_csv('../outputs/blazhko_list'+end+'.csv')
BE_candidates_ztf_chi = category_analysis(blazhko_can_ztfchi, fits, periodogr, ztf_data, dataLINEAR, 'ztf_periodogram',id_list=None,parameter=None, value=None)

In [ ]:
blazhko_analyzer = BE_candidates_ztf_chi.get_save_data()
end = 'ztf_chi_final'
blazhko_analyzer.to_csv("../outputs/blazhko_list"+end+".csv", index=False)

---

In [ ]:
BE_candidates_linear_chi = category_analysis(blazhko_can, fits, periodogr, ztf_data, dataLINEAR, 'linear_chi',id_list=None,parameter='ChiType', value='L')

In [ ]:
blazhko_analyzer = BE_candidates_linear_chi.get_save_data()
end = 'linear_chi'
blazhko_analyzer.to_csv("../outputs/blazhko_list"+end+".csv", index=False)

In [ ]:
blazhko_canlinearchi = pd.read_csv('../outputs/blazhko_list'+end+'.csv')
BE_candidates_linear_chi = category_analysis(blazhko_canlinearchi, fits, periodogr, ztf_data, dataLINEAR, 'ztf_periodogram',id_list=None,parameter=None, value=None)

In [ ]:
blazhko_analyzer = BE_candidates_linear_chi.get_save_data()
end = 'linear_chi_final'
blazhko_analyzer.to_csv("../outputs/blazhko_list"+end+".csv", index=False)

---

In [ ]:
BE_candidates_other = category_analysis(be_other, fits, periodogr, ztf_data, dataLINEAR, 'other')

In [ ]:
blazhko_analyzer = BE_candidates_other.get_save_data()
end = 'other'
blazhko_analyzer.to_csv("../outputs/blazhko_list"+end+".csv", index=False)

In [ ]:
blazhko_can_other = pd.read_csv('../outputs/blazhko_list'+end+'.csv')
BE_candidates_other = category_analysis(blazhko_can_other, fits, periodogr, ztf_data, dataLINEAR, 'ztf_periodogram',id_list=None,parameter=None, value=None)

In [ ]:
blazhko_analyzer = BE_candidates_other.get_save_data()
end = 'other_final'
blazhko_analyzer.to_csv("../outputs/blazhko_list"+end+".csv", index=False)

---

In [ ]:
blazhko_can['LINEAR id'].to_numpy()[240]

In [ ]:
BE_candidates_experimental = category_analysis(blazhko_can, fits, periodogr, ztf_data, dataLINEAR, end='bla',id_list=[61011,999528,1308565])